<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">
 
# Neural Networks with Keras
 
_Author: Adam Jones, PhD (San Francisco)_
 
---

Adapted from Jason Brownlee's [article](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)

**Dependencies**:
- SciPy (including NumPy)
- Keras\*
    - As well as a Keras backend (Theano, TensorFlow, etc.)

\* For 'simple' installation: `$ pip install --upgrade tensorflow` otherwise, see [here](https://www.tensorflow.org/install/).

From the [Keras docs](https://keras.io/):
> "Keras is a *high-level* neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling ***fast experimentation***."

This tutorial is intended to serve as a *brief*\* introduction to Keras, through an example similar to the types pipelines you've probably already seen. As such it can be broken down into these familiar steps:

- [Load Data](#load)
- [Define/Instantiate Model](#def-model)
- [Fit Model](#fit-model)
- [Evaluate Model Fit](#eval-model)

\*There will be relatively little code, as we'll move through the materials pretty slowly in order to emphasize clarity.

<a id="load"></a>
## Part 1. Load Data
Anytime you're working with stochastic processes, it's a good idea to see a random number seed for reproducibility of your results. Having added variation due to these processes can make optimization that much more laborious and difficult.

In [1]:
# Import toolkits
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn import model_selection

# Fix random seed for reproducibility
np.random.seed(7)

/home/adam/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


This dataset comes from the UCI Machine Learning repository (see data dictionary [here](./datasets/pima-indians-diabetes.txt)).

It was collected from the medical records of female Pima Indians (21 and older) and indicates whether they were diagnosed as diabetic within five years of collection. 

For this notebook, we'll use several different medical predictors as our independent variables and one outcome as our dependent variable: the onset of diabetes. Possible independent variables will include the number of pregnancies, BMI, insulin level, age, etc. *It will be necessary to follow the link to the data dictionary above for the complete description.*

In [2]:
pima_data = pd.read_csv('../datasets/pima-indians-diabetes.csv', header=None)

# Count observations
print('Number of observations: {}'.format(pima_data.shape[0]))
# Count features
print('Number of features: {}'.format(pima_data.shape[1]-1))

Number of observations: 768
Number of features: 8


**Let's explore the data a bit before training...**

As we can see, this dataset contains eight input variables and one output variable (the last column), corresponding to 768 observations (patients) total. Once loaded we can split the dataset into input variables (X) and the output class variable (Y).

Our label, onset of diabetes as 1 or not as 0, implies that this is a *binary classification problem*.

These are all already in numeric format, which makes it easy to use directly with neural networks that expect numerical input and output values.

In [3]:
# Split into input (X) and output (Y) variables
X = pima_data.iloc[:,0:6]
y = pima_data.iloc[:,8] # Onset of diabetes (True/False)

feature_num = X.shape[1]

As we've seen countless times by now, we always want to **split our data** into *training* and *testing* groups in order to be reasonably confident in our model's performance with new data.

In [4]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=7)

<a id="def-model"></a>
## Part 2. Define/Instantiate Model

Models in Keras are defined as a sequence of layers.

When we create a `Sequential` model, we add layers one at a time until our network is complete.

One key thing is to ensure the input layer is expecting the right number of inputs. This can be specified when creating the first layer via the `input_dim` argument (e.g. `8` for 8 input variables).

In [5]:
# Create model
model = Sequential() # Instantiate model ('Sequential' = A linear stack of layers)
model.add(Dense(12, input_dim=feature_num, activation='relu')) # 'Dense' = Fully-connected layer)
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

>For the purposed of today's class, we will skip over the role of optimizers, but activation functions, metric functions, and loss functions should be pretty familiar concepts by now. You are encouraged to research each of these concepts as part of the exercise later in this notebook.

Notice that we are using the rectifier (‘relu‘) activation function on the first two layers and the sigmoid function in the output layer. We've learned about the sigmoid function, and indeed, it used to be the case that sigmoid activation functions were preferred for all layers. More recently (~2011), however, better performance has been found with the [rectifier activation function](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)), which has become the most popular activation function for deep neural networks. We still use a sigmoid on the output layer to ensure our network output is between 0 and 1 and maps easily to a probability of class 1.

Determining the best shape for your model can involve quite a bit of trial and error. But, in general, your network needs to be large enough to represent the complexity of the problem at hand, and ideally not larger than that (for speed's sake).

In [6]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Compile the model
Now that the model is defined, it must be compiled.

Compiling uses the 'back-end' (i.e. the efficient numerical libraries under the covers, such as Theano or TensorFlow), to automatically choose the best way representation of this operation given your specific hardware, such as CPU, GPU, or distributed.

#### Visualize the model
It can be useful to routinely visualize our model, if only for QA purposes. Keras has a built-in method that works pretty well, but requires that Pydot be installed\* to do so.

*To install: `$pip install pydot`

In our [current example](model.png), we can see that we're using a fully-connected network structure consisting of with three layers.

We can specify the number of neurons in each layer, as well as the activation function, in the process of creating our model.

In [7]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

The first layer has 12 neurons and expects $N_{features}$ input variables. The second hidden layer has 8 neurons and finally, the output layer has 1 neuron to make our binary categorical prediction (onset of diabetes or not).

<a id="fit-model"></a>
## Part 3. Fit Model

We can train or fit our model on our loaded data by calling the `fit()` function on the model.

The training process will run for a fixed number of iterations through the dataset called `epochs`, that we must specify using the `nepochs` argument. We can also set the number of instances that are evaluated before a weight update in the network is performed, called the batch size and set using the `batch_size` argument.

For this problem, we will run for a small number of iterations (150) and use a relatively small batch size of 10. Again, these are often selected experimentally by trial and error.

This is where most of the 'work' happens.

In [8]:
# Fit the model
model.fit(X_train, y_train, epochs=150, batch_size=10)

Epoch 1/150
576/576 [==============================] - 0s 715us/step - loss: 4.2289 - acc: 0.6181
Epoch 2/150
576/576 [==============================] - 0s 159us/step - loss: 2.6335 - acc: 0.5868
Epoch 3/150
576/576 [==============================] - 0s 151us/step - loss: 0.8496 - acc: 0.6094
Epoch 4/150
576/576 [==============================] - 0s 158us/step - loss: 0.7465 - acc: 0.6181
Epoch 5/150
576/576 [==============================] - 0s 152us/step - loss: 0.7375 - acc: 0.5816
Epoch 6/150
576/576 [==============================] - 0s 165us/step - loss: 0.6885 - acc: 0.6094
Epoch 7/150
576/576 [==============================] - 0s 156us/step - loss: 0.6722 - acc: 0.6337
Epoch 8/150
576/576 [==============================] - 0s 143us/step - loss: 0.6689 - acc: 0.6406
Epoch 9/150
576/576 [==============================] - 0s 159us/step - loss: 0.6523 - acc: 0.6476
Epoch 10/150
576/576 [==============================] - 0s 157us/step - loss: 0.6409 - acc: 0.6354
Epoch 11/150
576/57

576/576 [==============================] - 0s 163us/step - loss: 0.5345 - acc: 0.7240
Epoch 84/150
576/576 [==============================] - 0s 140us/step - loss: 0.5300 - acc: 0.7326
Epoch 85/150
576/576 [==============================] - 0s 153us/step - loss: 0.5472 - acc: 0.7274
Epoch 86/150
576/576 [==============================] - 0s 155us/step - loss: 0.5411 - acc: 0.7292
Epoch 87/150
576/576 [==============================] - 0s 146us/step - loss: 0.5316 - acc: 0.7188
Epoch 88/150
576/576 [==============================] - 0s 158us/step - loss: 0.5213 - acc: 0.7361
Epoch 89/150
576/576 [==============================] - 0s 165us/step - loss: 0.5285 - acc: 0.7187
Epoch 90/150
576/576 [==============================] - 0s 152us/step - loss: 0.5360 - acc: 0.7309
Epoch 91/150
576/576 [==============================] - 0s 160us/step - loss: 0.5326 - acc: 0.7361
Epoch 92/150
576/576 [==============================] - 0s 146us/step - loss: 0.5321 - acc: 0.7118
Epoch 93/150
576/576 [=

<a id="eval-model"></a>
## Part 4. Evaluate Model Fit

In [9]:
# Evaluate the model
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

192/192 [==============================] - 0s 264us/step

acc: 71.35%


You can evaluate your model on your training dataset using the `evaluate()` function on your model and pass it the data used to evaluate the model's fit (i.e. 'testing data').

As we've seen before, `predict()` can be used to generate a prediction for each input.

In [10]:
# Calculate predictions
predictions = model.predict(X).round().astype(int)

# Convert to more human-readable format
target_key = {1: 'Diabetic', 0: 'Not Diabetic'}
print(pd.Series(np.squeeze(predictions)).map(target_key).head())

0        Diabetic
1    Not Diabetic
2    Not Diabetic
3    Not Diabetic
4        Diabetic
dtype: object


<a id="exercise"></a>
## Part 5. Exercise

Now it's your turn. In the cell below:

1. **Train-test split**
You probably already noticed one problem with our code- we're using the same data for training and testing (*oh no!*). Fix this. See what happens to our expectations regarding how the model will perform with future data.

2. **Feature selection**
We are only using a small number of the features available. Take a look at the data dictionary to understand the different variables contained in this dataset. Then, try changing which features are included based on your new understanding of the data. See if your score improves.

3. **Parameter tuning**
Optimization algorithms helps us to minimize (or maximize) loss (or reward) functions. Keras has a [variety](https://keras.io/optimizers/) of optimizers available for you to use. Take a look at a few that are available, try to find the right one for our model, and apply it.
Similarly, see if you can implement a different [metric function](https://keras.io/metrics/) or [loss function](https://keras.io/losses/).


In [11]:
## Create your first MLP in Keras
# Fix random seed for reproducibility
np.random.seed(7)

# Split into input (X) and output (Y) variables
X = pima_data.iloc[:,0:1] 
y = pima_data.iloc[:,8] # Onset of diabetes (True/False)
feature_num = X.shape[1] # Number of features in our model

# Create model
model = Sequential()
model.add(Dense(12, input_dim=feature_num, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='mean_absolute_error', optimizer='sgd', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=50, batch_size=10, verbose=1)

# Evaluate the model
scores = model.evaluate(X, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/50
768/768 [==============================] - 0s 326us/step - loss: 0.4796 - acc: 0.6510
Epoch 2/50
768/768 [==============================] - 0s 127us/step - loss: 0.4613 - acc: 0.6510
Epoch 3/50
768/768 [==============================] - 0s 117us/step - loss: 0.4433 - acc: 0.6510
Epoch 4/50
768/768 [==============================] - 0s 118us/step - loss: 0.4268 - acc: 0.6510
Epoch 5/50
768/768 [==============================] - 0s 107us/step - loss: 0.4130 - acc: 0.6510
Epoch 6/50
768/768 [==============================] - 0s 116us/step - loss: 0.4021 - acc: 0.6510
Epoch 7/50
768/768 [==============================] - 0s 120us/step - loss: 0.3935 - acc: 0.6510
Epoch 8/50
768/768 [==============================] - 0s 123us/step - loss: 0.3868 - acc: 0.6510
Epoch 9/50
768/768 [==============================] - 0s 111us/step - loss: 0.3814 - acc: 0.6510
Epoch 10/50
768/768 [==============================] - 0s 124us/step - loss: 0.3771 - acc: 0.6510
Epoch 11/50
768/768 [========

In [12]:
## Create your first MLP in Keras
# Fix random seed for reproducibility
np.random.seed(7)

# Split into input (X) and output (Y) variables
X = pima_data.iloc[:,0:8] 
y = pima_data.iloc[:,8] # Onset of diabetes (True/False)

# Create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, y, epochs=150, batch_size=10, verbose=0)

# Evaluate the model
scores = model.evaluate(X, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

768/768 [==============================] - 0s 100us/step

acc: 77.86%


<a id="topic-review"></a>
## Summary
---

Specifically, you saw all of the five essential steps  tocreate a neural network or deep learning model using Keras, including:

- How to define/instantiate neural network in Keras.
- How to compile a Keras model using the efficient numerical backend.
- How to train a model on data.
- How to evaluate a model's fit on data.

**Any further questions?**